### PyTorch MLP

In [72]:
from sklearn.datasets import make_blobs
import torch 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np

In [73]:
batch_size = 8

In [74]:
def blob_label(y, label, loc): # assign labels
    
    target = np.copy(y)
    
    for l in loc:
        target[y == l] = label
        
    return target

In [75]:
x_train, y_train = make_blobs(n_samples=40, n_features=2, cluster_std=1.5, shuffle=True)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(blob_label(y_train, 0, [0]))
y_train = torch.FloatTensor(blob_label(y_train, 1, [1,2,3]))

x_test, y_test = make_blobs(n_samples=10, n_features=2, cluster_std=1.5, shuffle=True)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(blob_label(y_test, 0, [0]))
y_test = torch.FloatTensor(blob_label(y_test, 1, [1,2,3]))

In [76]:
class TabularDataset(Dataset):
    
    """Generic dataset class for tabular data.
    """
    
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [77]:
dataset = TabularDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)    

In [78]:
class Feedforward(torch.nn.Module):

    def __init__(self, input_size, hidden_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()        
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [79]:
model = Feedforward(2, 10)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [80]:
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

Test loss before training 0.33175915479660034


In [85]:
model.train() # Set the model to 'train' mode
epoch = 20

for epoch in range(epoch):    
    
    for batch in dataloader:
        
        x_batch, y_batch = batch[0], batch[1]
    
        # Forward pass
        optimizer.zero_grad()      # Remove gradients from prev passes
        y_pred = model(x_train)    # Compute loss
        loss = criterion(y_pred.squeeze(), y_train)
   
        # Backward pass
        loss.backward() # Backprop
        optimizer.step() # Update params
        
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    

Epoch 0: train loss: 0.04980311542749405
Epoch 1: train loss: 0.04970930889248848
Epoch 2: train loss: 0.049616117030382156
Epoch 3: train loss: 0.049523480236530304
Epoch 4: train loss: 0.04943143576383591
Epoch 5: train loss: 0.04933997988700867
Epoch 6: train loss: 0.049249060451984406
Epoch 7: train loss: 0.04915871098637581
Epoch 8: train loss: 0.049068938940763474
Epoch 9: train loss: 0.04897967353463173
Epoch 10: train loss: 0.04889099299907684
Epoch 11: train loss: 0.04880281910300255
Epoch 12: train loss: 0.04871521145105362
Epoch 13: train loss: 0.048628102988004684
Epoch 14: train loss: 0.048541538417339325
Epoch 15: train loss: 0.048455484211444855
Epoch 16: train loss: 0.04836992546916008
Epoch 17: train loss: 0.04828489199280739
Epoch 18: train loss: 0.048200350254774094
Epoch 19: train loss: 0.048116303980350494


In [86]:
model.eval() # Set the model to evaluate mode
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 

print('Test loss after Training' , after_train.item())

Test loss after Training 0.03833035007119179
